# Experiment 003: 3-Model Ensemble (XGBoost + LightGBM + CatBoost)

Following evaluator's top priority recommendation:
1. Use the mature feature set from exp_002 (56 features)
2. Train XGBoost, LightGBM, and CatBoost with 5-fold CV
3. Simple averaging of probabilities
4. Expected improvement: 0.5-1% over single XGBoost

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
import warnings
warnings.filterwarnings('ignore')

# Load data
train = pd.read_csv('/home/data/train.csv')
test = pd.read_csv('/home/data/test.csv')

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")

Train shape: (8693, 14)
Test shape: (4277, 13)


In [2]:
# Copy feature engineering from exp_002
def advanced_feature_engineering(df):
    df = df.copy()
    df['Group'] = df['PassengerId'].apply(lambda x: int(x.split('_')[0]))
    df['PassengerNum'] = df['PassengerId'].apply(lambda x: int(x.split('_')[1]))
    df['Deck'] = df['Cabin'].apply(lambda x: x.split('/')[0] if pd.notna(x) else np.nan)
    df['CabinNum'] = df['Cabin'].apply(lambda x: int(x.split('/')[1]) if pd.notna(x) else np.nan)
    df['Side'] = df['Cabin'].apply(lambda x: x.split('/')[2] if pd.notna(x) else np.nan)
    df['Surname'] = df['Name'].apply(lambda x: x.split()[-1] if pd.notna(x) else np.nan)
    return df

train = advanced_feature_engineering(train)
test = advanced_feature_engineering(test)

# GroupSize
all_data = pd.concat([train[['Group']], test[['Group']]], ignore_index=True)
group_sizes = all_data['Group'].value_counts().to_dict()
for df in [train, test]:
    df['GroupSize'] = df['Group'].map(group_sizes)
    df['Solo'] = (df['GroupSize'] == 1).astype(int)

print("Basic features created")

Basic features created


In [3]:
# Group-based imputation
def group_based_imputation(train_df, test_df):
    combined = pd.concat([train_df, test_df], ignore_index=True)
    
    group_homeplanet = combined.groupby('Group')['HomePlanet'].apply(
        lambda x: x.mode()[0] if len(x.mode()) > 0 else np.nan).to_dict()
    group_deck = combined.groupby('Group')['Deck'].apply(
        lambda x: x.mode()[0] if len(x.mode()) > 0 else np.nan).to_dict()
    group_side = combined.groupby('Group')['Side'].apply(
        lambda x: x.mode()[0] if len(x.mode()) > 0 else np.nan).to_dict()
    
    for df in [train_df, test_df]:
        for col, mapping in [('HomePlanet', group_homeplanet), ('Deck', group_deck), ('Side', group_side)]:
            mask = df[col].isna()
            df.loc[mask, col] = df.loc[mask, 'Group'].map(mapping)
    return train_df, test_df

train, test = group_based_imputation(train, test)

# Remaining imputation
def impute_remaining(df):
    df = df.copy()
    spending_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    for col in spending_cols:
        mask = (df['CryoSleep'] == True) & (df[col].isna())
        df.loc[mask, col] = 0
    
    for col in ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side']:
        if df[col].isna().any():
            df[col] = df[col].fillna(df[col].mode()[0])
    
    for col in ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'CabinNum']:
        if df[col].isna().any():
            df[col] = df[col].fillna(df[col].median())
    return df

train = impute_remaining(train)
test = impute_remaining(test)
print("Imputation complete")

Imputation complete


In [4]:
# Spending features
def create_spending_features(df):
    df = df.copy()
    spending_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    
    df['TotalSpent'] = df[spending_cols].sum(axis=1)
    for col in spending_cols:
        df[f'{col}_ratio'] = df[col] / (df['TotalSpent'] + 1)
    
    df['LuxurySpent'] = df['Spa'] + df['VRDeck'] + df['RoomService']
    df['BasicSpent'] = df['FoodCourt'] + df['ShoppingMall']
    df['LuxuryRatio'] = df['LuxurySpent'] / (df['TotalSpent'] + 1)
    df['SpentPerAge'] = df['TotalSpent'] / (df['Age'] + 1)
    df['SpendingBin'] = pd.cut(df['TotalSpent'], bins=[-1, 0, 500, 2000, float('inf')], labels=[0, 1, 2, 3]).astype(int)
    
    for col in spending_cols:
        df[f'{col}_spent'] = (df[col] > 0).astype(int)
    
    for col in spending_cols + ['TotalSpent', 'LuxurySpent', 'BasicSpent']:
        df[f'{col}_log'] = np.log1p(df[col])
    
    df['NumSpendingCategories'] = sum(df[f'{col}_spent'] for col in spending_cols)
    df['Spa_VRDeck_RoomService'] = df['Spa'] + df['VRDeck'] + df['RoomService']
    df['FoodCourt_RoomService'] = df['FoodCourt'] + df['RoomService']
    return df

train = create_spending_features(train)
test = create_spending_features(test)
print("Spending features created")

Spending features created


In [5]:
# Interaction features
def create_interaction_features(df):
    df = df.copy()
    df['CryoSleep_HomePlanet'] = df['CryoSleep'].astype(str) + '_' + df['HomePlanet'].astype(str)
    df['CryoSleep_Destination'] = df['CryoSleep'].astype(str) + '_' + df['Destination'].astype(str)
    df['Deck_Side'] = df['Deck'].astype(str) + '_' + df['Side'].astype(str)
    df['HomePlanet_Destination'] = df['HomePlanet'].astype(str) + '_' + df['Destination'].astype(str)
    df['AgeGroup'] = pd.cut(df['Age'], bins=[0, 12, 17, 25, 40, 60, 100], 
                            labels=['Child', 'Teen', 'YoungAdult', 'Adult', 'MiddleAge', 'Senior'])
    df['AgeGroup_CryoSleep'] = df['AgeGroup'].astype(str) + '_' + df['CryoSleep'].astype(str)
    df['VIP_HomePlanet'] = df['VIP'].astype(str) + '_' + df['HomePlanet'].astype(str)
    
    df['IsChild'] = (df['Age'] <= 12).astype(int)
    df['IsTeen'] = ((df['Age'] > 12) & (df['Age'] <= 17)).astype(int)
    df['IsYoungAdult'] = ((df['Age'] > 17) & (df['Age'] <= 25)).astype(int)
    df['IsAdult'] = ((df['Age'] > 25) & (df['Age'] <= 60)).astype(int)
    df['IsSenior'] = (df['Age'] > 60).astype(int)
    return df

train = create_interaction_features(train)
test = create_interaction_features(test)
print("Interaction features created")

Interaction features created


In [6]:
# Encode categorical variables
cat_cols_to_encode = [
    'HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side',
    'CryoSleep_HomePlanet', 'CryoSleep_Destination', 'Deck_Side',
    'HomePlanet_Destination', 'AgeGroup', 'AgeGroup_CryoSleep', 'VIP_HomePlanet'
]

for col in cat_cols_to_encode:
    le = LabelEncoder()
    combined = pd.concat([train[col].astype(str), test[col].astype(str)])
    le.fit(combined)
    train[col + '_enc'] = le.transform(train[col].astype(str))
    test[col + '_enc'] = le.transform(test[col].astype(str))

print(f"Encoded {len(cat_cols_to_encode)} categorical columns")

Encoded 13 categorical columns


In [7]:
# Define features (same as exp_002)
feature_cols = [
    'HomePlanet_enc', 'CryoSleep_enc', 'Destination_enc', 'VIP_enc', 'Deck_enc', 'Side_enc',
    'CryoSleep_HomePlanet_enc', 'CryoSleep_Destination_enc', 'Deck_Side_enc',
    'HomePlanet_Destination_enc', 'AgeGroup_enc', 'AgeGroup_CryoSleep_enc', 'VIP_HomePlanet_enc',
    'Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
    'Group', 'PassengerNum', 'CabinNum', 'GroupSize', 'Solo',
    'TotalSpent', 'LuxurySpent', 'BasicSpent', 'LuxuryRatio', 'SpentPerAge', 'SpendingBin',
    'NumSpendingCategories', 'Spa_VRDeck_RoomService', 'FoodCourt_RoomService',
    'RoomService_ratio', 'FoodCourt_ratio', 'ShoppingMall_ratio', 'Spa_ratio', 'VRDeck_ratio',
    'RoomService_spent', 'FoodCourt_spent', 'ShoppingMall_spent', 'Spa_spent', 'VRDeck_spent',
    'RoomService_log', 'FoodCourt_log', 'ShoppingMall_log', 'Spa_log', 'VRDeck_log',
    'TotalSpent_log', 'LuxurySpent_log', 'BasicSpent_log',
    'IsChild', 'IsTeen', 'IsYoungAdult', 'IsAdult', 'IsSenior'
]

X = train[feature_cols].values
y = train['Transported'].astype(int).values
X_test = test[feature_cols].values

print(f"X shape: {X.shape}, Features: {len(feature_cols)}")

X shape: (8693, 56), Features: 56


In [8]:
# Model hyperparameters from strategy
xgb_params = {
    'max_depth': 5,
    'learning_rate': 0.067,
    'n_estimators': 850,
    'reg_lambda': 3.06,
    'reg_alpha': 4.58,
    'colsample_bytree': 0.92,
    'subsample': 0.95,
    'random_state': 42,
    'n_jobs': -1,
    'eval_metric': 'logloss'
}

lgb_params = {
    'num_leaves': 330,
    'learning_rate': 0.087,
    'n_estimators': 739,
    'feature_fraction': 0.66,
    'bagging_fraction': 0.87,
    'bagging_freq': 1,
    'lambda_l1': 6.18,
    'lambda_l2': 0.01,
    'random_state': 42,
    'verbose': -1
}

cat_params = {
    'depth': 6,
    'learning_rate': 0.05,
    'iterations': 1000,
    'l2_leaf_reg': 3.0,
    'random_seed': 42,
    'verbose': False
}

print("Model parameters defined")

Model parameters defined


In [ ]:
# 5-fold CV for all three models
n_folds = 5
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

# OOF predictions for each model
oof_xgb = np.zeros(len(X))
oof_lgb = np.zeros(len(X))
oof_cat = np.zeros(len(X))

# Test predictions for each model
test_xgb = np.zeros(len(X_test))
test_lgb = np.zeros(len(X_test))
test_cat = np.zeros(len(X_test))

# Fold scores
xgb_scores = []
lgb_scores = []
cat_scores = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    # XGBoost
    model_xgb = xgb.XGBClassifier(**xgb_params)
    model_xgb.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    oof_xgb[val_idx] = model_xgb.predict_proba(X_val)[:, 1]
    test_xgb += model_xgb.predict_proba(X_test)[:, 1] / n_folds
    xgb_acc = accuracy_score(y_val, (oof_xgb[val_idx] >= 0.5).astype(int))
    xgb_scores.append(xgb_acc)
    
    # LightGBM
    model_lgb = lgb.LGBMClassifier(**lgb_params)
    model_lgb.fit(X_train, y_train, eval_set=[(X_val, y_val)])
    oof_lgb[val_idx] = model_lgb.predict_proba(X_val)[:, 1]
    test_lgb += model_lgb.predict_proba(X_test)[:, 1] / n_folds
    lgb_acc = accuracy_score(y_val, (oof_lgb[val_idx] >= 0.5).astype(int))
    lgb_scores.append(lgb_acc)
    
    # CatBoost
    model_cat = CatBoostClassifier(**cat_params)
    model_cat.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=100)
    oof_cat[val_idx] = model_cat.predict_proba(X_val)[:, 1]
    test_cat += model_cat.predict_proba(X_test)[:, 1] / n_folds
    cat_acc = accuracy_score(y_val, (oof_cat[val_idx] >= 0.5).astype(int))
    cat_scores.append(cat_acc)
    
    print(f"Fold {fold+1}: XGB={xgb_acc:.5f}, LGB={lgb_acc:.5f}, CAT={cat_acc:.5f}")

print(f"\nXGBoost Mean CV: {np.mean(xgb_scores):.5f} (+/- {np.std(xgb_scores):.5f})")
print(f"LightGBM Mean CV: {np.mean(lgb_scores):.5f} (+/- {np.std(lgb_scores):.5f})")
print(f"CatBoost Mean CV: {np.mean(cat_scores):.5f} (+/- {np.std(cat_scores):.5f})")

In [ ]:
# Ensemble: Simple averaging of probabilities
oof_ensemble = (oof_xgb + oof_lgb + oof_cat) / 3
test_ensemble = (test_xgb + test_lgb + test_cat) / 3

# Evaluate ensemble
ensemble_pred = (oof_ensemble >= 0.5).astype(int)
ensemble_acc = accuracy_score(y, ensemble_pred)

print(f"\n=== ENSEMBLE RESULTS ===")
print(f"Ensemble OOF Accuracy: {ensemble_acc:.5f}")

# Compare to individual models
xgb_oof_acc = accuracy_score(y, (oof_xgb >= 0.5).astype(int))
lgb_oof_acc = accuracy_score(y, (oof_lgb >= 0.5).astype(int))
cat_oof_acc = accuracy_score(y, (oof_cat >= 0.5).astype(int))

print(f"\nIndividual OOF Accuracies:")
print(f"  XGBoost:  {xgb_oof_acc:.5f}")
print(f"  LightGBM: {lgb_oof_acc:.5f}")
print(f"  CatBoost: {cat_oof_acc:.5f}")

# Compare to exp_002 baseline
exp002_acc = 0.80927
improvement = ensemble_acc - exp002_acc
print(f"\nImprovement over exp_002 (single XGBoost): {improvement:+.5f} ({improvement*100:+.2f}%)")

In [ ]:
# Create submission
test_binary = (test_ensemble >= 0.5).astype(bool)
submission = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Transported': test_binary
})

submission.to_csv('/home/submission/submission.csv', index=False)
print(f"Submission saved with {len(submission)} predictions")
print(f"Predicted transported rate: {test_binary.mean():.4f}")
print(submission.head())